<h1 style="color: cyan; text-align: center;">Projet de Natural Language Processing : Analyse de sentiments des films de la plateforme TMDB</h1>

# Introduction

Ce document présente les codes Python appliqués dans le cadre du projet de traitement automatique du langage. Il présente les différentes étapes des démarches effectuées en les décrivant succintement. Pour plus de détail concernant ce travail, son objectif et sa réalisation, veuillez vous référer au document **demarche_projet.pdf** fourni avec l'ensemble des documents au moment du rendu.

## <u>Objectifs :</u>

Ce projet vise à développer une application interactive en Python fournissant aux utilisateurs des recommandations de films à regarder en priorité. Ces recommandations ne sont pas basées sur les moyens mis par les sociétés de production pour en faire la publicité mais uniquement sur le ressenti des utilisateurs. Cette approche vise à recommander films, qui pourraient être moins mis en avant dans les catalogues, pour donner de nouvelles idées au utilisateurs lorsqu'ils ne savent pas quoi regarder.

Les notes affectées aux films, pour les classer en ordre de priorité à voir, seront calculées à partir d'une analyse de sentiments des avis déposés par les utilisateurs du site https://www.themoviedb.org. Une note par avis sera calculée puis une aggrégation sera effectuée pour créer un unique indicateur par film.


<font color="red">L'objectif à la fin sera aussi de comparer nos notes déterminées sur les avis à celles déposées par les utilisateurs si il y en a eu. En effet, il arrive que les utilisateurs déposent pour un film un avis écrit et une note sur 5.

Ces avis pourraient en quelques sortes nous servir de base de test pour vérifier la fiabilité de nos analyses.
</font>

# Imports des librairies

<font color="red">Le package imdb est à importer via le code ci-dessous :</font>

```bash
pip install git+https://github.com/santhoshse7en/imdb
```

In [1]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
from datetime import datetime
import json
import re
import pandas as pd
from pprint import pprint
from collections import Counter
from nltk.corpus import stopwords
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from flask import Flask, render_template, request

# Collecte et stockage des données

Plusieurs techniques ont été explorées pour construire notre base de données *(voir rapport pdf pour plus d'explications)*:

- L'API de TMDB : "https://developers.themoviedb.org/3/"
- Scraping du site : imdb.com (via la librairie python : beautifulsoup4)
- Utilisation de la librairie python : imdbpy

La méthode que nous avons retenue est celle à partir de l'API de IMDb. 

Avant de travailler avec l'API, il est nécessaire de créer une clé d'accès à l'API du site de themoviedb. Celle que nous utilisons est stockée dans le fichier *clefAPI.json*.

- Appel de la clé API de TMDB

In [2]:
with open('clefAPI.json', 'r') as config_file:
    config = json.load(config_file)

api_key = config['tmdb']['api_key']

- Collecte des données

Nous commençons par nous restreindre à une certaine période pour les dates de sorties de film afin de ne pas avoir à traiter l'ensemble de la base de données. Cela nous évitera un temps de traitement trop important.

De plus, on on ne sélectionne que les films présents sur le site qui ont au minimum un avis déposé par un utilisateur.

In [3]:
def get_movies_between_dates(start_date, end_date, api_key): # trouve les films sortis entre 2 dates
    base_url = "https://api.themoviedb.org/3/discover/movie"
    movies = []
    page = 1
    total_pages = 1

    while page <= total_pages:
        url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR&page={page}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            total_pages = data['total_pages']
            movies.extend([(movie['id'], movie['title']) for movie in data['results']])
            page += 1
        else:
            print("Erreur lors de la requête :", response.status_code)
            break
    return movies

def get_movie_reviews(movie_id, api_key): # retourne les avis d'un film
    url = f"https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['results']
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
        return None

def filter_movies_with_reviews(movies, api_key): # filtre les films qui ont des avis
    movies_with_reviews = []
    for movie_id, movie_title in movies:
        reviews = get_movie_reviews(movie_id, api_key)
        if reviews and len(reviews) > 0:
            movies_with_reviews.append((movie_id, movie_title))
    return movies_with_reviews

In [ ]:
start_date = "2022-01-01"
end_date = "2022-03-31"
print(f"Recherche des films sortis entre le {start_date} et le {end_date}")

movies = get_movies_between_dates(start_date, end_date, api_key)
print(f"Nombre de films trouvés : {len(movies)}")

movies_with_reviews = filter_movies_with_reviews(movies, api_key)
print(f"Nombre de films avec des avis de {start_date} à {end_date} : {len(movies_with_reviews)}")

## Structuration des données

Maintenant que nous disposons des données brutes pour les films ayant un ou des avis sur une certaine période, il est nécessaire de structurer les données afin de faciliter leur utilisation ensuite.

In [5]:
# Fonction qui etourne les informations sur un film et ses avis :

def get_movie_details_and_reviews(movie_id, api_key): 
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    movie_details = response.json() if response.status_code == 200 else {}
    reviews_url = f"{base_url}{movie_id}/reviews?api_key={api_key}"
    reviews_response = requests.get(reviews_url)
    reviews_data = reviews_response.json() if reviews_response.status_code == 200 else {}
    return movie_details, reviews_data

# print(get_movie_details_and_reviews('tt0111161', api_key))

On applique cette fonction à l'ensemble des films qui rentrent dans nos conditions.

In [ ]:
movies_details = {}

for movie_id, movie_title in movies_with_reviews:
    movie_details, reviews_data = get_movie_details_and_reviews(movie_id, api_key)
    movies_details[movie_id] = {
        'title': movie_title,
        'details': movie_details,
        'reviews': reviews_data
    }

Pour éviter de relancer à chaque fois les exécutions, dont les requêtes qui peuvent être assez longues, nous stockons la base créée dans un fichier *.json*. Il suffit simplement d'accéder à ce fichier par la suite sans faire de requêtes.

<font color="red">Revoir le nom du fichier</font>

In [ ]:
# with open('movies_details_3mois.json', 'w') as json_file: 
#    json.dump(movies_details, json_file, indent=4)

# Présentation des données

Avant de se lancer dans l'analyse de sentiments, nous allons d'abord présenter les données dont nous disposons.


# <font color="red">Tout remettre au propre: on relie à chaque fois le fichier json: non</font>

In [7]:
with open('movies_details_3mois.json', 'r') as json_file:
    movies_details = json.load(json_file)

pprint(movies_details)

{'245842': {'details': {'adult': False,
                        'backdrop_path': '/o3U4733mGsmieBeqs4Sw2ze6n53.jpg',
                        'belongs_to_collection': None,
                        'budget': 40500000,
                        'genres': [{'id': 14, 'name': 'Fantasy'},
                                   {'id': 12, 'name': 'Adventure'},
                                   {'id': 10751, 'name': 'Family'}],
                        'homepage': 'https://thekingsdaughtermovie.com/',
                        'id': 245842,
                        'imdb_id': 'tt2328678',
                        'original_language': 'en',
                        'original_title': "The King's Daughter",
                        'overview': "King Louis XIV's quest for immortality "
                                    'leads him to capture and steal a '
                                    "mermaid's life force, a move that is "
                                    'further complicated by his illegitimate '


In [8]:
num_films = len(movies_details)
print("Nombres de films:", num_films)

try:
    with open('movies_details_3mois.json', 'r') as file:
        movies_details = json.load(file)
    reviews_count = {}
    for movie in movies_details.values():
        num_reviews = len(movie['reviews']['results'])
        movie_id = movie['reviews']['id']
        reviews_count[movie_id] = num_reviews
    for movie_id, count in reviews_count.items():
        print(f"Film ID {movie_id}: Nombre d'avis = {count}")
except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

Nombres de films: 83
Film ID 414906: Nombre d'avis = 20
Film ID 675353: Nombre d'avis = 7
Film ID 585083: Nombre d'avis = 1
Film ID 760104: Nombre d'avis = 5
Film ID 508947: Nombre d'avis = 11
Film ID 406759: Nombre d'avis = 6
Film ID 818647: Nombre d'avis = 1
Film ID 335787: Nombre d'avis = 8
Film ID 646385: Nombre d'avis = 9
Film ID 629542: Nombre d'avis = 5
Film ID 545611: Nombre d'avis = 17
Film ID 526896: Nombre d'avis = 9
Film ID 752623: Nombre d'avis = 8
Film ID 774825: Nombre d'avis = 1
Film ID 505026: Nombre d'avis = 9
Film ID 557946: Nombre d'avis = 2
Film ID 628900: Nombre d'avis = 3
Film ID 924482: Nombre d'avis = 1
Film ID 698508: Nombre d'avis = 1
Film ID 823625: Nombre d'avis = 1
Film ID 619979: Nombre d'avis = 3
Film ID 579974: Nombre d'avis = 5
Film ID 696806: Nombre d'avis = 8
Film ID 799876: Nombre d'avis = 5
Film ID 522016: Nombre d'avis = 6
Film ID 854867: Nombre d'avis = 1
Film ID 763285: Nombre d'avis = 8
Film ID 787752: Nombre d'avis = 3
Film ID 833339: Nombre d

Regardons par exemple la détail des informations pour un film; ici, le film dont l'ID est **646385**.

In [9]:
fichier_json = 'movies_details_3mois.json'
movie_id_to_check = 646385

try:
    with open(fichier_json, 'r') as file:
        movies_details = json.load(file)

    selected_movie = movies_details[str(movie_id_to_check)]
    reviews = selected_movie['reviews']['results']
    
    for review in reviews:
        print(f"Auteur : {review['author']}")
        print(f"Rating : {review['author_details']['rating']}")
        print(f"Contenu : {review['content']}\n")

except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

Auteur : garethmb
Rating : None
Contenu : It is hard to believe it has been 25 years since “Scream” premiered and in doing so; brought new life into the slasher genre which had become stale due to an abundance of direct to DVD and cable releases combined with many uninspired and stale theatrical releases.

The movie was a massive success and launched a very successful trilogy and made the Ghostface killer and many moments from the series Pop Culture mainstays.

After a hiatus; “Scream 4” arrived and continued the series and in 2022; “Scream” has arrived which continues the story to the eager anticipation of fans.

When a young girl named Tara (Jenna Ortega); is brutally attacked in a fashion very similar to the Ghostface Killer; her sister Sam (Melissa Barrera); returns to Woodsboro to be by her side while Tara recuperates.

Sam is joined by her boyfriend Richie (Jack Quaid), who has no idea about the history of Woodsboro nor the popular “Stab” film series that is based on the events o

<font color="red">Rajouter quelques stats globales</font>

# Traitement des données

Avant d'analyser le sens des avis laissés par les utilisateurs, il est nécessaire d'effectuer certeins traitements.

- On va d'abord créer une fonction clean_avis qui va nous permettre de nettoyer les commentaires récupérés sur TMDB. On va supprimer les caractères spéciaux, les mots vides et on va mettre tous les mots en minuscule. Pour cela on va s'aider de la librairie nltk et de son module stopwords et de la librairie re.

- Ensuite on va construire une fonction qui va nous permettre de compter le nombre d'occurence de chaque mot dans les commentaires. On utilisera notamment la fonction Counter de la librairie collections.

<font color="red">PENSER A RETIRER LES MOTS INUTILES, IL Y EN A ENCORE APRES LE CLEAN</font>


In [10]:
stop_words = set(stopwords.words('english'))

def clean_avis(text):
    mots_a_retirer = ['film', 'movie']
    all_stopwords = stop_words.union(set(mots_a_retirer))
    all_stopwords = list(stop_words) + mots_a_retirer
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    words = [word for word in text.split() if word not in all_stopwords]
    return words

def analyse_word_freq(movie_reviews):
    all_words = []
    for review in movie_reviews:
        all_words.extend(clean_avis(review['content']))
    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(10)
    return most_common_words

word_freq_film = {}

for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    most_common_words = analyse_word_freq(reviews)
    word_freq_film[movie_id] = most_common_words

for movie_id, frequencies in word_freq_film.items():
    print(f"{movies_details[movie_id]['title']}\nMots les plus fréquents {frequencies}\n")

The Batman
Mots les plus fréquents [('batman', 123), ('like', 32), ('riddler', 23), ('also', 22), ('even', 18), ('pattinson', 17), ('bruce', 17), ('dark', 17), ('much', 17), ('best', 16)]

Sonic the Hedgehog 2
Mots les plus fréquents [('sonic', 20), ('first', 8), ('carrey', 8), ('one', 7), ('jim', 7), ('hedgehog', 7), ('robotnik', 6), ('well', 6), ('review', 6), ('fun', 6)]

Hotel Transylvania: Transformania
Mots les plus fréquents [('hotel', 2), ('transylvania', 2), ('transformania', 2), ('review', 2), ('lot', 1), ('obstacles', 1), ('relating', 1), ('release', 1), ('viewing', 1), ('understand', 1)]

X
Mots les plus fréquents [('x', 6), ('horror', 6), ('west', 5), ('story', 4), ('goth', 4), ('pearl', 4), ('makes', 4), ('gore', 3), ('mia', 3), ('interesting', 3)]

Turning Red
Mots les plus fréquents [('red', 11), ('pixar', 10), ('one', 9), ('turning', 8), ('mei', 8), ('animation', 6), ('panda', 6), ('puberty', 6), ('going', 6), ('characters', 6)]

Moonfall
Mots les plus fréquents [('one

# Mise en place des éléments calculés pour présenter les films

<font color="red">L'objectif de cette partie est de réaliser les calculs d'indicateurs et d'autres contenus qui seront affichés sur les pages individuelles des films.</font>

## WordClouds

Pour commencer, nous réalisons des WordClouds pour que l'utilisateur puisse, en cliquant sur un film, disposer des thèmes principaux qui sont évoqués lorsque l'on parle d'un film.

In [ ]:
def create_wordcloud(text, movie_id):
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stop_words, 
                min_font_size = 10).generate(' '.join(text))
    wordcloud.to_file(f"static/wordclouds/{movie_id}.png")
    
for movie_id, movie_details in movies_details.items():
    reviews = movie_details['reviews']['results']
    review_text = [review['content'] for review in reviews]
    cleaned_text = []
    for text in review_text:
        cleaned_text.extend(clean_avis(text))
    create_wordcloud(cleaned_text, movie_id)

## Mise en place des informations par catégorie de film

L'objectif de l'application est de présenter une liste de film, ordonnée par ordre de priorité, à voir. Nous avons fait le choix de demander à l'utilisateur de renseigner un genre dans l'application afin de séparer les films en fonction de ses envies.

- Pour voir les genres présents dans notre base de données :

In [11]:
def extract_unique_genres(file_path):
    with open(file_path, 'r') as file:
        movies_details = json.load(file)

    unique_genres = set()
    for movie in movies_details.values():
        genres = movie.get('details', {}).get('genres', [])
        for genre in genres:
            unique_genres.add(genre['name'])

    return unique_genres

genres_uniques = extract_unique_genres('movies_details_3mois.json')
print("Genres uniques :", genres_uniques)

Genres uniques : {'Thriller', 'Documentary', 'Mystery', 'Family', 'Romance', 'Drama', 'Animation', 'Science Fiction', 'History', 'Adventure', 'Crime', 'Fantasy', 'Horror', 'Action', 'Comedy', 'Music', 'TV Movie', 'Western'}


## Analyse des sentiments des avis

Afin de réaliser un classement des films à voir, il est nécessaire de résumer chaque film par un indicateur. Celui que nous avons choisi est la moyenne de l'ensemble des notes déterminées à partir des avis (puisque chaque film de la base possède au moins un avis).

In [12]:
def analyse_sentiment(text): # fonction d'analyse de sentiments
    return TextBlob(text).sentiment.polarity

def calculate_movie_polarity(movie_reviews): # calcul la polarité moyenne de tous les avis d'un film
    total_polarity = 0
    num_reviews = len(movie_reviews)

    for review in movie_reviews:
        total_polarity += analyse_sentiment(review['content'])

    return total_polarity / num_reviews if num_reviews > 0 else 0

def genre_top_movies(file_path):
    with open(file_path, 'r') as file:
        movies_details = json.load(file)

    genre_movies = {genre: [] for genre in genres_uniques}

    for movie_id, movie in movies_details.items():
        movie_polarity = calculate_movie_polarity(movie['reviews']['results'])
        movie_genres = [genre['name'] for genre in movie['details']['genres']]

        for genre in movie_genres:
            if genre in genre_movies:
                genre_movies[genre].append((movie_id, movie['title'], movie_polarity))

    top_movies_per_genre = {}

    for genre, movies in genre_movies.items():
        top_movies = sorted(movies, key=lambda x: x[2], reverse=True)[:3]
        top_movies_per_genre[genre] = top_movies

    return top_movies_per_genre
    # renvoie un dictionnaire où chaque clé est un genre et chaque valeur est une liste de tuples 
    # contenant l'ID, le titre et la polarité moyenne des 3 meilleurs films de ce genre
    

top_movies = genre_top_movies('movies_details_3mois.json')
print(top_movies)

{'Thriller': [('787752', 'Fresh', 0.32541526374859714), ('924482', 'The Ledge', 0.31666666666666665), ('680829', 'Master', 0.3145652173913044)], 'Documentary': [('396194', 'Ennio', 0.2652173913043478), ('940075', 'Embrace the Panda: Making Turning Red', 0.2594612068965517), ('939356', 'Marvel Studios Assembled: The Making of Eternals', 0.25)], 'Mystery': [('680829', 'Master', 0.3145652173913044), ('646385', 'Scream', 0.2464593214865157), ('799876', 'The Outfit', 0.19018307296568165)], 'Family': [('675353', 'Sonic the Hedgehog 2', 0.26728324200689235), ('585083', 'Hotel Transylvania: Transformania', 0.21666666666666665), ('629542', 'The Bad Guys', 0.21539164177736833)], 'Romance': [('803736', 'Hey! Sinamika', 0.262291280148423), ('680860', 'I Want You Back', 0.18571428571428572), ('615904', 'Marry Me', 0.17956410415318158)], 'Drama': [('661925', 'Softie', 0.5171675943104515), ('787752', 'Fresh', 0.32541526374859714), ('680829', 'Master', 0.3145652173913044)], 'Animation': [('931034', 'C

# Création de l'application finale

NOus finissons par créer un support visuel pour présenter nos résultats.

In [50]:
app = Flask(__name__)

def polarity_to_stars(polarity):
    return round(polarity * 10)

base_url = "https://image.tmdb.org/t/p/original"

for movie_id, details in movies_details.items():
    backdrop_path = details["details"].get("backdrop_path")
    if backdrop_path:
        details["image_url"] = base_url + backdrop_path

app.jinja_env.globals.update(polarity_to_stars=polarity_to_stars)

@app.route('/', methods=['GET', 'POST'])
def index():
    genres = list(top_movies.keys())
    selected_genre = request.form.get('genre')
    top_movies_genre = top_movies.get(selected_genre, []) if selected_genre else []

    movies_with_images = []
    for movie_id, movie_title, polarity in top_movies_genre:
        image_url = movies_details[movie_id].get("image_url", "")
        movies_with_images.append((movie_id, movie_title, polarity, image_url))

    return render_template('index2.html', genres=genres, top_movies_genre=movies_with_images, selected_genre=selected_genre)

@app.route('/film/<int:movie_id>')
def film(movie_id):
    with open('movies_details_3mois.json', 'r') as file:
        movies_details = json.load(file)
    
    movie_details = movies_details.get(str(movie_id), {})
    reviews = movie_details.get('reviews', {}).get('results', [])[:5]

    return render_template('film_details.html', 
                        movie_details=movie_details, 
                        reviews=reviews)
    
@app.template_filter('join_genres')
def join_genres(genres_list):
    return ', '.join([genre['name'] for genre in genres_list])

    
@app.template_filter('format_currency')
def format_currency(value):
    if value == 0:
        return "ya walou"
    else:
        return "${:,.2f}".format(value)


if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2023 00:44:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2023 00:44:26] "GET /static/wp.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2023 00:44:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2023 00:44:28] "GET /static/wp.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Dec/2023 00:44:29] "GET /film/931034 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2023 00:44:29] "GET /phBYksmk15iwt5NqnAfzkhwbdTX.jpg HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2023 00:44:35] "GET /film/585083 HTTP/1.1" 200 -
127.0.0.1 - - [22/Dec/2023 00:44:35] "GET /teCy1egGQa0y8ULJvlrDHQKnxBL.jpg HTTP/1.1" 404 -
